## Dataset Split

In [28]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import re

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/srv/idp-radio-1'

In [ ]:
#!wget http://download.cs.stanford.edu/deep/CheXpert-v1.0-small.zip


In [47]:
load_dotenv(find_dotenv())
chestxray14_folder = Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY"))
chexpert_folder = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY"))
print(chestxray14_folder)
print(chexpert_folder)

data/chestxray14/scale_256
data/chexpert/preprocessed-256-crop


In [51]:
chestxray14_labels = pd.read_csv(chestxray14_folder / "meta/data/labels.csv")
chexpert_train_labels = pd.read_csv(chexpert_folder / "train.csv")
#chexpert_test_labels = pd.read_csv(chexpert_folder / "test.csv")

In [ ]:
chestxray14_labels.head()

In [4]:
chexpert_train_labels.head()

,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1
1,1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,...,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,2
2,2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,...,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
3,3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,...,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
4,4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3


In [ ]:
chexpert_test_labels.head()

In [ ]:
!ls data/chexpert/full/CheXpert-v1.0-small/train/patient28746/

In [ ]:
def get_patient_id (row):
    patient_folder = re.findall(r"patient[0-9]+", row["Path"])
    if len(patient_folder) > 0:
        patient_number = re.findall(r"[0-9]+", patient_folder[0])
    if len(patient_number) > 0:
        return (patient_number[0])

In [ ]:
chexpert_train_labels["patient_id"] = chexpert_train_labels.apply(get_patient_id, axis = 1)
chexpert_test_labels["patient_id"] = chexpert_test_labels.apply(get_patient_id, axis = 1)


In [54]:
# remove all samples where the parent label is negative
print(len(chexpert_train_labels))
filterp = chexpert_train_labels[['Lung Opacity', 'Consolidation']].sum(axis=1) > 0 
filtered = chexpert_train_labels[filterp]
print(len(filtered))
filtered.to_csv(chexpert_folder / "filtered_dependencies.csv")

223414
95418


In [ ]:
chexpert_train_labels["patient_id"].value_counts()

In [ ]:
chexpert_test_labels["patient_id"].value_counts()

In [ ]:
#!mkdir ~/idp-radio-1/data/dev_dataset
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train
!ls ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train

In [ ]:
#chexpert_train_labels.to_csv(chexpert_folder / "train.csv")
#chexpert_test_labels.to_csv(chexpert_folder / "test.csv")

In [ ]:
chexpert_train_labels.columns

In [ ]:
no_finding_mask_train = chexpert_train_labels["No Finding"] == 1.0
#no_finding_mask_test = chexpert_test_labels["No Finding"] == 1.0

In [ ]:
finding_columns = ['Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture']
replacement_values = {finding: 0.0 for finding in finding_columns}

chexpert_train_labels[no_finding_mask_train] = chexpert_train_labels[no_finding_mask_train].fillna(value=replacement_values)
#chexpert_test_labels[no_finding_mask_test] = chexpert_test_labels[no_finding_mask_test].fillna(value=replacement_values)

In [ ]:
chexpert_train_labels["No Finding"].value_counts()

In [ ]:
no_finding_mask_train.value_counts()

In [ ]:
chexpert_train_labels[no_finding_mask_train]

In [ ]:
chexpert_train_labels.to_csv(chexpert_folder / "nofinding_train.csv")
#chexpert_test_labels.to_csv(chexpert_folder / "nofinding_test.csv")